In [1]:
import pandas as pd
import numpy as np
import math, sys, logging
logging.basicConfig()
log = logging.getLogger('dynamic')
log.setLevel(logging.INFO)

## Shortest common supersequence

Ok, so I was confused here, I thought a supersequence was one that directly contained its subsequences intact, so like how "ABCDE" contains "ABC" and "CDE". But the definition of subsequence also includes "ACE" as a subsequence, because it contains the same elements, the same number of times, in the same order. This is the difference between subsequences and substrings.

In [2]:
a = {0: ('a', 'b'), 1: ('c', 'd')}
print(a[1])
df = pd.DataFrame(a)
print(df)
ssl = pd.DataFrame(-1, index=range(6), columns=range(5), dtype=int)
ssl
ssl.rank(1)
ssl.iat[3,4] = 1
ssl[:] = -1
ssl

sss = 'abcd'
print(sss[1:3])

('c', 'd')
   0  1
0  a  c
1  b  d
bc


In [3]:
def shortest_superseq(s1, s2):
    l1, l2 = len(s1), len(s2)
    seqs = pd.DataFrame(-1,
        index=range(l1+1), columns=range(l2+1),
        dtype=int)
    _ss(s1, s2, l1, l2, seqs)
    return seqs
    
def _ss(s1, s2, i, j, seqs):
    prev = seqs.iat[i,j]
    if prev != -1:
        return prev
    if i == 0:
        ret = j
    elif j == 0:
        ret = i
    elif s1[i-1] == s2[j-1]:
        ret = _ss(s1, s2, i-1, j-1, seqs) + 1
    else:
        r1 = _ss(s1, s2, i-1, j, seqs) + 1
        r2 = _ss(s1, s2, i, j-1, seqs) + 1
        ret = min(r1, r2)
    seqs.iat[i,j] = ret
    return ret
        

In [4]:
def find_supersequences(seqs, s1, s2):
    l1, l2 = len(s1), len(s2)
    sl = []
    msl = seqs.iat[l1, l2]
    _fss(seqs, s1, s2, l1, l2, sl, '')
    return sl
    
def _fss(seqs, s1, s2, i, j, sl, s):
    p = seqs.iat[i,j]
    if i == 0:
        sl.append(s2[0:j]+s)
    elif j == 0:
        sl.append(s1[0:i]+s)
    else:
        if s1[i-1] == s2[j-1]:
            _fss(seqs, s1, s2, i-1, j-1, sl, s1[i-1]+s)
        else:
            if seqs.iat[i,j] - 1 == seqs.iat[i-1,j]:
                _fss(seqs, s1, s2, i-1, j, sl, s1[i-1]+s)
            if seqs.iat[i,j] - 1 == seqs.iat[i,j-1]:
                _fss(seqs, s1, s2, i, j-1, sl, s2[j-1]+s)
    

In [5]:
s1 = 'ABCBDAB'
s2 = 'BDCABA'
seqs = shortest_superseq(s1, s2)
seqslabeled = seqs.copy()
seqslabeled.loc[1:,('str')] = list(s1)
seqslabeled = seqslabeled.T
seqslabeled.loc[1:-1,('str')] = list(s2)
print(seqslabeled)
seqlist = find_supersequences(seqs, s1, s2)
seqlist


       0   1   2   3   4   5   6   7  str
0     -1   1  -1   3  -1  -1  -1  -1  NaN
1      1   2   2   3   4  -1  -1  -1    B
2      2   3   3   4   5   5  -1  -1    D
3      3   4   4   4   5   6  -1  -1    C
4     -1   4   5   5   6   7   7  -1    A
5     -1  -1  -1  -1   6   7  -1   8    B
6     -1  -1  -1  -1  -1  -1   8   9    A
str  NaN   A   B   C   B   D   A   B  NaN


['ABDCABDAB', 'ABDCBDABA', 'ABCBDCABA']

## Word break problem

Given a dictionary and a condensed string of non-whitespace characters, split the string up into combinations of words in the dictionary. Both determine if it is possible, and find all the satisfying strings.

In [6]:
def word_break(dct, st):
    last = len(st)
    words = []
    if _wb(dct, st, 0, 1, last, words):
        return words
    else:
        return None

def _wb(dct, st, start, end, last, words):
    word = st[start:end]
    if (word in dct):
        log.debug('adding word: %s', word)
        words.append(word)
        if end == last:
            return True
        else:
            return _wb(dct, st, end, end+1, last, words)
    else:
        if end == last:
            return False
        else:
            return _wb(dct, st, start, end+1, last, words)

### Dynamic approach

Given that this is from a dynamic programming list, it makes sense that this would exist. Once we identify any word or group of words from one portion of the string, and we know those are all the words in that substring, then we can do the same for the remaining string separately, and each combination of breakups of each of those two strings becomes the total combinations for all breaks that have a separation at that point. Each individual calculation is also a microcosm of the original problem, which is to say, the algorithm that solves the problem generally would be applied as-is to each substring.

Ex: "pickaxeatone" -> if separated into "pickaxe" and "atone", we can look at each separate string in isolation. If you look at "atone" and find "atone", "a tone", and "at one", then you know that the total combination involving that substring will be those combinations crossed with the combinations made from the remaining substring.

Therefore, what I want to do is constract a 2D array, where each place in the outer array represents the starting substring index, and each item in the inner array at that point is the list of words beginning at that point ("a" , "at", and "atone" in the prior example, if we start at "a"). Then to find the other combinations, you move forward in the outer array by the length of that string. Do that recursives and get all your combinations.

In [6]:
def word_break_complete(dct, st):
    last = len(st)
    return _wbc(dct, st, 0, last):

def _wbc(dct, st, start, last):
    for i in range(start+1, last):
        word = st[start:i]
        if (word in dct):
            words = []
            log.debug('adding word: %s', word)
            words.append(word)
            if end == last:
                return True
            else:
                return _wbc(dct, st, end, end+1, last)

In [7]:
log.setLevel(logging.DEBUG)
dct = { "this", "th", "is", "famous", "Word", "break", "b","r", "e", "a", "k", "br", "bre", "brea", "ak", "problem" }
st = "Wordbreakproblem"
print(word_break(dct, st))

DEBUG:dynamic:adding word: Word
DEBUG:dynamic:adding word: b
DEBUG:dynamic:adding word: r
DEBUG:dynamic:adding word: e
DEBUG:dynamic:adding word: a
DEBUG:dynamic:adding word: k
DEBUG:dynamic:adding word: problem


['Word', 'b', 'r', 'e', 'a', 'k', 'problem']
